### Notebook Version
**- ver6. 9/26, 0.95734 :** 
    - Aug 수정(Peter), 224, Model:B4, DenseCrossEntropy(nn.Module)
    - 5-Fold, Random-No,개별 Softmax -> 합 softmax
    
**- ver7. 9/27, 0.95912 :** 
    - nn.CrossEntropyLoss()

**- ver8. 9/28,0.94741 :**
    - Aug 수정(Akash)
    - Model:B5, Linear+ ReLU + Dropout
    - Train Info Graph
**- ver11. 9/28,000 :**
    - Ver7로 복귀
    - B0~B7 평가 (7->0):  7:0.183,6:0.1880,5: 0.194,4 0.160,3 0.189,2 0.176, 1 0.179,0 0.169    
    
**- ver13. 10/3,0.95860 :**
    - Focal Loss
    
**- ver13./ver3 10/4,0.95860 :**
    - Focal Loss 1,2,3
    - model 4ea(efficientnet, resnet50,densenet, inception)
**- ver14./ver3 10/4,0.95860 :**
    - cutmix, focal loss3

In [1]:
model_name="effinet"
version_info="v14"
def get_model(model_name):
    if model_name=="effinet": return EffiNet()
    if model_name=="resnet": return Resnet50()    
    if model_name=="densenet": return Densenet()    
    if model_name=="effinet7": return EffiNet7()    

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore') 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [3]:
train=pd.read_csv("../input/train-fold/train_fold.csv")
test=pd.read_csv("/kaggle/input/plant-pathology-2020-fgvc7/test.csv")
sample_submission=pd.read_csv("/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv")

In [4]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16035 sha256=7dd79960ea301ccafa6b352f5b4187be18199261716ceb81c22b7959b30da580
  Stored in directory: /root/.cache/pip/wheels/b7/cc/0d/41d384b0071c6f46e542aded5f8571700ace4f1eb3f1591c29
Successfully built efficientnet-pytorch


In [5]:
from PIL import Image
class ClassificationLoader:
    def __init__(self, image_paths, targets, resize, augmentations=None):
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        image = Image.open(self.image_paths[item])
        targets = self.targets[item]

        image = np.array(image)
        if self.augmentations:
            image = self.augmentations(image=image)["image"]
        return {
            "img":torch.tensor(image, dtype=torch.float),
            "tar":torch.tensor(targets, dtype=torch.long)
        }

In [6]:
from torch.utils.data.dataset import Dataset
class CutMix(Dataset):
    def __init__(self, dataset, num_class, num_mix=1, beta=1., prob=0.5):
        self.dataset = dataset
        self.num_class = num_class
        self.num_mix = num_mix
        self.beta = beta
        self.prob = prob
        
    def rand_bbox(self, size, lam):
        if len(size) == 4:
            W = size[2]
            H = size[3]
        elif len(size) == 3:
            W = size[1]
            H = size[2]
        else:
            raise Exception

        cut_rat = np.sqrt(1. - lam)
        cut_w = np.int(W * cut_rat)
        cut_h = np.int(H * cut_rat)

        # uniform
        cx = np.random.randint(W)
        cy = np.random.randint(H)

        bbx1 = np.clip(cx - cut_w // 2, 0, W)
        bby1 = np.clip(cy - cut_h // 2, 0, H)
        bbx2 = np.clip(cx + cut_w // 2, 0, W)
        bby2 = np.clip(cy + cut_h // 2, 0, H)

        return bbx1, bby1, bbx2, bby2

    def onehot(self,size, target):
        vec = torch.zeros(size, dtype=torch.float32)
        vec[target] = 1.
        return vec
    
    def __getitem__(self, index):
        data=self.dataset[index]
        img, lb = data["img"],data["tar"]
        lb_onehot = self.onehot(self.num_class, lb)

        for _ in range(self.num_mix):
            r = np.random.rand(1)
            if self.beta <= 0 or r > self.prob:
                continue

            # generate mixed sample
            lam = np.random.beta(self.beta, self.beta)
            rand_index = random.choice(range(len(self)))

            data=self.dataset[rand_index]
            img2, lb2 = data["img"],data["tar"]
            lb2_onehot = self.onehot(self.num_class, lb2)

            bbx1, bby1, bbx2, bby2 = self.rand_bbox(img.size(), lam)
            img[:, bbx1:bbx2, bby1:bby2] = img2[:, bbx1:bbx2, bby1:bby2]
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
            lb_onehot = lb_onehot * lam + lb2_onehot * (1. - lam)

        return {
            "img":torch.tensor(img, dtype=torch.float),
            "tar":torch.tensor(lb_onehot, dtype=torch.long)
        }

    def __len__(self):
        return len(self.dataset)

In [7]:
# EfficientNet
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
class EffiNet(nn.Module):
    def __init__(self):
        super(EffiNet, self).__init__()
        self.base_model = EfficientNet.from_pretrained("efficientnet-b4")
        num_ftrs = self.base_model._fc.in_features
        self.base_model._fc = nn.Linear(num_ftrs,4, bias = True)
        
    def forward(self, image):
        out = self.base_model(image)
        return out
# Resnet50
import torchvision
class Resnet50(nn.Module):
    def __init__(self,model_no):
        super(Resnet50, self).__init__()
        self.base_model = torchvision.models.resnext50_32x4d(pretrained=True)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(num_ftrs,4, bias = True)
        
    def forward(self, image):
        out = self.base_model(image)
        return out    
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
class EffiNet7(nn.Module):
    def __init__(self):
        super(EffiNet7, self).__init__()
        self.base_model = EfficientNet.from_pretrained("efficientnet-b7")
        num_ftrs = self.base_model._fc.in_features
        self.base_model._fc = nn.Linear(num_ftrs,4, bias = True)
        
    def forward(self, image):
        out = self.base_model(image)
        return out 
# densenet
import torchvision
class Densenet(nn.Module):
    def __init__(self,model_no):
        super(Densenet, self).__init__()
        self.base_model = torchvision.models.densenet161(pretrained=True)
        num_ftrs = self.base_model.classifier.in_features
        self.base_model.classifier = nn.Linear(num_ftrs,4, bias = True)
        
    def forward(self, image):
        out = self.base_model(image)
        return out    

In [8]:
import albumentations as A
from albumentations.pytorch import ToTensor, ToTensorV2
SIZE=224
import random

class Utils:
    def __init__():
        pass
    def get_aug(mode="train"):
        if mode=="Nor":
            aug=A.Compose([
                ToTensor(),
            ])
        elif mode =="train":
            aug=A.Compose([
                #A.RandomResizedCrop(height=SIZE, width=SIZE, p=1.0),
                A.Flip(),
                A.ShiftScaleRotate(rotate_limit=1.0, p=0.8),

                # Pixels
                A.OneOf([
                    A.IAAEmboss(p=1.0),
                    A.IAASharpen(p=1.0),
                    A.Blur(p=1.0),
                ], p=0.5),

                # Affine
                A.OneOf([
                    A.ElasticTransform(p=1.0),
                    A.IAAPiecewiseAffine(p=1.0)
                ], p=0.5),

                A.Normalize(p=1.0),
                ToTensor(),
            ])
        else:
            aug=A.Compose([
                A.Normalize(p=1.0),
                ToTensor(),
            ])

        return aug 

In [9]:
# ## My Image cumix test
# import matplotlib.pyplot as plt

# train_df=train[train.kfold!=0].reset_index(drop=True)
# imgs=train_df.image_id.values.tolist()
# path="../input/plant-images-224-224/"
# train_imgs=[path+file+".png" for file in imgs]
# train_aug=Utils.get_aug("Nor")
# train_tar=train_df.category.values
# train_dataset=ClassificationLoader(
#     image_paths=train_imgs,targets=train_tar,resize=None,augmentations=train_aug
# )
# CutMix_train_dataloader = CutMix(train_dataset, 
#                           num_class=4, 
#                           beta=1.0, 
#                           prob=0.999, 
#                           num_mix=1)
# train_loader=torch.utils.data.DataLoader(
#     CutMix_train_dataloader,batch_size=128,num_workers=4,shuffle=True)

# cnt=0
# for img,tar in train_loader:
#     for i in range(128):
#         img2=img[i].permute(1,2,0)
#         tar2=tar[i].numpy()
#         #print(tar2)        
#         plt.imshow(np.uint(img2*256))
#         plt.figure()
#         cnt+=1
#     if cnt>10:
#         break

In [10]:
#clcarwin / focal_loss_pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int,long)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [11]:
# My Focal Loss
class FocalLoss2(nn.Module):
    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss2, self).__init__()
        self.gamma = gamma
        self.eps = eps
    def fl_onehot(self,index,classes,tar):
        y_onehot = torch.FloatTensor(index, classes).to("cuda")
        y_onehot.zero_()
        y_onehot.scatter_(1, tar, 1)
        return (y_onehot)        
    def forward(self, inputs, targets):
        y = self.fl_onehot(inputs.size()[0],inputs.size()[1],targets.view(-1,1))
        logit = F.softmax(inputs, dim=-1)
        logit = logit.clamp(self.eps, 1. - self.eps)

        loss = -1 * y * torch.log(logit) # cross entropy
        loss = loss * (1 - logit) ** self.gamma # focal loss

        return loss.mean()  

In [12]:
# My Focal Loss
class FocalLoss3(nn.Module):# for cutmix
    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss3, self).__init__()
        self.gamma = gamma
        self.eps = eps
    def fl_onehot(self,index,classes,tar):
        y_onehot = torch.FloatTensor(index, classes).to("cuda")
        y_onehot.zero_()
        y_onehot.scatter_(1, tar, 1)
        return (y_onehot)        
    def forward(self, inputs, targets):
        logit = F.softmax(inputs, dim=-1)
        logit = logit.clamp(self.eps, 1. - self.eps)

        loss = -1 * targets * torch.log(logit) # cross entropy
        loss = loss * (1 - logit) ** self.gamma # focal loss

        return loss.mean()  

In [13]:
import torch.nn.functional as F
class DenseCrossEntropy(nn.Module):

    def __init__(self):
        super(DenseCrossEntropy, self).__init__()
        
        
    def forward(self, logits, labels):
        logits = logits.float()
        labels = labels.float()
        
        logprobs = F.log_softmax(logits, dim=-1)
        
        loss = -labels * logprobs
        loss = loss.sum(-1)

        return loss.mean()

In [14]:
# from sklearn.metrics import accuracy_score

# labels_for_acc = []
# out_for_acc = []
# for i in range(5):
#     d1=np.random.randint(0,10,(10,4))
#     d2=np.random.randint(0,10,(10,4))
#     if len(labels_for_acc)==0:
#         labels_for_acc=d1
#         out_for_acc=d2
#     else:
#         labels_for_acc=np.vstack((labels_for_acc, d1))
#         out_for_acc=np.vstack((out_for_acc, d2))
        
# print("data concat",labels_for_acc)   
# print(labels_for_acc.shape)
# print(out_for_acc.shape)
# print(accuracy_score(labels_for_acc.argmax(1), out_for_acc.argmax(1)))
# (labels_for_acc.argmax(1)==out_for_acc.argmax(1)).sum()/labels_for_acc.shape[0]

data concat [[3 1 0 2]
 [8 2 6 6]
 [1 5 0 9]
 [6 5 5 6]
 [0 7 2 5]
 [5 8 8 7]
 [8 1 1 4]
 [3 5 1 9]
 [8 5 6 6]
 [4 7 3 1]
 [2 3 2 8]
 [4 6 8 5]
 [1 3 7 0]
 [8 7 6 7]
 [1 0 1 3]
 [5 3 9 9]
 [6 7 9 2]
 [3 8 8 8]
 [9 5 8 5]
 [1 1 3 1]
 [4 9 0 9]
 [9 7 7 9]
 [4 6 4 3]
 [1 9 4 6]
 [6 7 8 0]
 [7 2 9 1]
 [4 5 1 6]
 [3 3 5 7]
 [5 1 9 1]
 [0 0 8 1]
 [0 3 8 5]
 [5 3 4 2]
 [4 5 8 7]
 [6 2 2 7]
 [8 7 2 4]
 [1 5 3 6]
 [0 4 1 2]
 [0 8 3 2]
 [6 1 5 8]
 [3 8 2 5]
 [9 8 2 9]
 [4 7 0 1]
 [9 3 0 6]
 [3 5 7 1]
 [7 6 1 5]
 [4 3 7 4]
 [1 0 0 6]
 [2 0 9 4]
 [2 1 4 4]
 [7 8 7 7]]
(50, 4)
(50, 4)
0.28


0.28

In [31]:
## Engine
from sklearn.metrics import accuracy_score
class Engine:
    def __init__(self,model,optimizer,device):
        self.model=model
        self.optimizer=optimizer
        self.device=device
    
    def loss_fn(self,targets,outputs):
        #return DenseCrossEntropy()(outputs,targets)
        #return nn.CrossEntropyLoss()(outputs,targets)
        #return FocalLoss2()(outputs,targets)
        return FocalLoss3()(outputs,targets)
    
    def train(self,data_loader):
        preds_for_acc = []
        labels_for_acc = []
        self.model.train()
        final_loss=0
        for data in data_loader:
            self.optimizer.zero_grad()
            inputs=data["img"].to(self.device)
            targets=data["tar"].to(self.device)
            outputs=self.model(inputs)
            loss=self.loss_fn(targets,outputs)
            loss.backward()
            self.optimizer.step()
            final_loss += loss.item()
            ## Acc Check
            #labels_for_acc = np.concatenate((labels_for_acc, targets.cpu().numpy()), 0)
            #preds_for_acc = np.concatenate((preds_for_acc, np.argmax(outputs.cpu().detach().numpy(), 1)), 0)
            if len(labels_for_acc)==0:
                labels_for_acc = targets.cpu().numpy()
                preds_for_acc = outputs.cpu().detach().numpy()
            else:
                labels_for_acc=np.vstack((labels_for_acc,targets.cpu().numpy()))
                preds_for_acc=np.vstack((preds_for_acc,outputs.cpu().detach().numpy()))
        accuracy = np.uint(labels_for_acc.argmax(1)==preds_for_acc.argmax(1)).sum()/labels_for_acc.shape[0]
        return final_loss/len(data_loader),accuracy
    
    def validate(self,data_loader):
        preds_for_acc = []
        labels_for_acc = []
        self.model.eval()
        final_loss=0
        for data in data_loader:
            inputs=data["img"].to(self.device)
            targets=data["tar"].to(self.device)
            with torch.no_grad():
                outputs=self.model(inputs)
                loss=self.loss_fn(targets,outputs)
                final_loss += loss.item()
            ## Acc Check
            #labels_for_acc = np.concatenate((labels_for_acc, targets.cpu().numpy()), 0)
            #preds_for_acc = np.concatenate((preds_for_acc, np.argmax(outputs.cpu().detach().numpy(), 1)), 0)
            if len(labels_for_acc)==0:
                labels_for_acc = targets.cpu().numpy()
                preds_for_acc = outputs.cpu().detach().numpy()
            else:
                labels_for_acc=np.vstack((labels_for_acc,targets.cpu().numpy()))
                preds_for_acc=np.vstack((preds_for_acc,outputs.cpu().detach().numpy()))
        accuracy = np.uint(labels_for_acc.argmax(1)==preds_for_acc.argmax(1)).sum()/labels_for_acc.shape[0]
        return final_loss/len(data_loader),accuracy
    
    def predict(self,data_loader):
        self.model.eval()
        final_predictions = []
        for data in data_loader:
            inputs=data["img"].to(self.device)
            predictions = self.model(inputs)
            predictions = predictions.cpu()
            final_predictions.append(predictions.detach().numpy())
        return final_predictions

In [32]:
train_info=dict()
valid_info=dict()
def ready_train_info(fold):
    key=f"Fold[{fold}]_Acc"
    train_info[key]=[]
    key=f"Fold[{fold}]_loss"
    train_info[key]=[]    
    key=f"Fold[{fold}]_Acc"
    valid_info[key]=[]
    key=f"Fold[{fold}]_loss"
    valid_info[key]=[]     
    
def add_train_info(fold, train_acc,train_loss,valid_acc,valid_loss):
    key=f"Fold[{fold}]_Acc"
    train_info[key].append(train_acc)
    key=f"Fold[{fold}]_loss"
    train_info[key].append(train_loss)
    key=f"Fold[{fold}]_Acc"
    valid_info[key].append(valid_acc)
    key=f"Fold[{fold}]_loss"
    valid_info[key].append(valid_loss)  

In [33]:
bs_train=16
bs_valid=12

import datetime
def loop_train(fold=0):
    ready_train_info(fold)
    train_df=train[train.kfold!=fold].reset_index(drop=True)#[0:32]
    valid_df=train[train.kfold==fold].reset_index(drop=True)#[0:32]

    imgs=train_df.image_id.values.tolist()
    path="../input/plant-images-224-224/"
    train_imgs=[path+file+".png" for file in imgs]
    train_aug=Utils.get_aug("train")
    #train_tar=train_df[["healthy","multiple_diseases","rust","scab"]].values
    train_tar=train_df.category.values
    train_dataset=ClassificationLoader(
        image_paths=train_imgs,targets=train_tar,resize=None,augmentations=train_aug
    )
    CutMix_train_dataloader = CutMix(train_dataset, 
                          num_class=4, 
                          beta=1.0, 
                          prob=0.999, 
                          num_mix=1)
    train_loader=torch.utils.data.DataLoader(
        CutMix_train_dataloader,batch_size=bs_train,num_workers=4,shuffle=True
    )
    
    imgs=valid_df.image_id.values.tolist()
    path="../input/plant-images-224-224/"
    valid_imgs=[path+file+".png" for file in imgs]
    valid_aug=Utils.get_aug("valid")
    #valid_tar=valid_df[["healthy","multiple_diseases","rust","scab"]].values
    valid_tar=valid_df.category.values
    valid_dataset=ClassificationLoader(
        image_paths=valid_imgs,targets=valid_tar,resize=None,augmentations=valid_aug
    )
    CutMix_valid_dataloader = CutMix(valid_dataset, 
                          num_class=4, 
                          beta=1.0, 
                          prob=0, 
                          num_mix=1)
    valid_loader=torch.utils.data.DataLoader(
        CutMix_valid_dataloader,batch_size=bs_valid,num_workers=4,shuffle=False
    )
    
    # Model,Optimizer, scheduler, engine
    model=get_model(model_name)
    
    device="cuda" if torch.cuda.is_available() else "cpu"
    model=model.to(device)
    optimizer=torch.optim.Adam(model.parameters(),lr=1e-4)
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,patience=3,threshold=1e-5,mode="min",verbose=True
    )

    engine=Engine(model,optimizer,device)
    best_loss=np.inf
    early_stopping=7#3
    early_stopping_cnt=0
    EPOCH=300
    for epoch in range(EPOCH):
        train_loss,train_acc=engine.train(train_loader)
        valid_loss,valid_acc=engine.validate(valid_loader)
        scheduler.step(valid_loss)
        
        # Add train Info
        add_train_info(fold,train_acc,train_loss,valid_acc,valid_loss)
        
        if valid_loss<best_loss :
            best_loss=valid_loss
            torch.save(model.state_dict(),f"model_fold_{fold}.bin")
            tm=datetime.datetime.now().strftime("%H:%M:%S")
            print(f"{tm}, fold={fold}, epoch={epoch}, train_loss={train_loss:.6f}, valid_loss={valid_loss:.6f}")    
            early_stopping_cnt=0
        else:
            early_stopping_cnt+=1
        if early_stopping_cnt>early_stopping:
            break

    print(f"fold={fold}, best val loss={best_loss}")

In [34]:
loop_train(fold=0)
loop_train(fold=1)
loop_train(fold=2)
loop_train(fold=3)
loop_train(fold=4)


Loaded pretrained weights for efficientnet-b4
10:18:49, fold=0, epoch=0, train_loss=0.088920, valid_loss=0.215254
10:20:22, fold=0, epoch=1, train_loss=0.041050, valid_loss=0.092403
10:21:49, fold=0, epoch=2, train_loss=0.018048, valid_loss=0.081035
10:23:19, fold=0, epoch=3, train_loss=0.018406, valid_loss=0.072008
10:24:48, fold=0, epoch=4, train_loss=0.020089, valid_loss=0.069869
10:26:21, fold=0, epoch=5, train_loss=0.014227, valid_loss=0.065572


KeyboardInterrupt: 

In [ ]:
bs_test=12
def predict(fold=0,model_no=0):    
    df=pd.read_csv("../input/plant-pathology-2020-fgvc7/test.csv")
    device="cuda" if torch.cuda.is_available() else "cpu"
    imgs=df.image_id.values.tolist()
    path="../input/plant-images-224-224/"
    test_imgs=[path+file+".png" for file in imgs]
    test_aug=Utils.get_aug("test")
    test_tar=np.zeros((len(imgs),4))
    test_dataset=ClassificationLoader(
        image_paths=test_imgs,targets=test_tar,resize=None,augmentations=test_aug
    )
    test_loader=torch.utils.data.DataLoader(
        test_dataset,batch_size=bs_test,num_workers=0,shuffle=False
    )

    model=get_model(model_name)
    model_save_path=f"./model_fold_{fold}.bin"
    model.load_state_dict(torch.load(model_save_path))
    model=model.to(device)

    engine=Engine(model,None,device)
    preds=engine.predict(test_loader)
    preds=np.vstack(preds)
    return preds

In [ ]:
from scipy.special import softmax

d1=predict(fold=0,model_no=4)
d2=predict(fold=0,model_no=4)
d3=predict(fold=0,model_no=4)
d4=predict(fold=0,model_no=4)
d5=predict(fold=0,model_no=4)

p1 = softmax(d1, axis=1)
p2 = softmax(d2, axis=1)
p3 = softmax(d3, axis=1)
p4 = softmax(d4, axis=1)
p5 = softmax(d5, axis=1)

In [ ]:
p=(p1+p2+p3+p4+p5)/5
p = softmax(p, axis=1)
submission_df = pd.read_csv("../input/plant-pathology-2020-fgvc7/sample_submission.csv")
submission_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = p
submission_df.to_csv(f'{version_info}_{model_name}_submission.csv', index = False)

In [ ]:
fold=0
plt.figure()
plt.ylim(0,1.5)

key=f"Fold[{fold}]_Acc"
sns.lineplot(list(range(len(train_info[key]))), train_info[key])
sns.lineplot(list(range(len(valid_info[key]))), valid_info[key])
plt.xlabel('Epoch')
plt.ylabel('Acc')
plt.legend(['Train','Val'])

In [ ]:
fold=0
plt.figure()
plt.ylim(0,15)

key=f"Fold[{fold}]_loss"
sns.lineplot(list(range(len(train_info[key]))), train_info[key])
sns.lineplot(list(range(len(valid_info[key]))), valid_info[key])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train','Val'])